In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import gc

# Initialize Mediapipe Pose module for pose detection and drawing utilities
mpPose = mp.solutions.pose
mpDraw = mp.solutions.drawing_utils

# Create a list to store names for each landmark (x, y, z, visibility)
name_points = []
lm_list = []
for i in range(33): # 33 landmarks for pose detection
    name_points.append(f"x{i+1}")
    name_points.append(f"y{i+1}")
    name_points.append(f"z{i+1}")
    name_points.append(f"visibility{i+1}")


# Function to extract pose landmarks from the results of pose detection
# Returns a list of pose points: x, y, z coordinates and visibility
def make_landmark_timestep(results):
    print(results.pose_world_landmarks.landmark)
    pose_points = []
    for id, lm in enumerate(results.pose_world_landmarks.landmark):
        pose_points.append(lm.x)
        pose_points.append(lm.y)
        pose_points.append(lm.z)
        pose_points.append(lm.visibility)
    return pose_points

# Function to draw pose landmarks on an image
# Draws both the pose connections and individual points (landmarks)
def draw_landmark_on_image(mpDraw, results, img):
    # Vẽ các đường nối
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Vẽ các điểm nút
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 10, (0, 0, 255), cv2.FILLED)
    return img
root_folder = "./resized_video/"


# Loop through all videos in the specified directory
for subdir, dirs, files in os.walk(root_folder):
    for video_file in files:
        if video_file.endswith(".mp4") or video_file.endswith(".avi"):  # Check if the file is a video
            video_path = os.path.join(subdir, video_file)
            label = os.path.basename(subdir)  # Use the directory name as the label

            cap = cv2.VideoCapture(video_path)
            name_video = video_file.split('.')[0]  # Use the video name as label
            print(name_video)

            lm_list = [name_points]
            # Process each frame in the video for pose detection
            with mpPose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    # Convert frame to RGB and process pose detection
                    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = pose.process(frameRGB)

                    if results.pose_world_landmarks:
                        # Store pose landmark data for each frame
                        lm = make_landmark_timestep(results)
                        lm_list.append(lm)
                        # Draw landmarks on the image
                        frame = draw_landmark_on_image(mpDraw, results, frame)

            cap.release()
            # Write in file csv
            df  = pd.DataFrame(lm_list)
            if not os.path.exists(f"landmarks/{label}"):
                os.makedirs(f"landmarks/{label}")
            df.to_csv(f"landmarks/{label}/"+ name_video + ".csv", index=False, header=False)
            # Clear memory after processing each video to prevent memory overload
            lm_list = []  # Reset landmark list
            gc.collect()  # Force garbage collection to free up unused memory

01502df8-593c-42d0-86ae-7cbd48d8741a


c:\Users\huuhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[x: -0.181855395
y: -0.661838
z: -0.0885369107
visibility: 0.999421
, x: -0.16099757
y: -0.688405
z: -0.110439688
visibility: 0.999526262
, x: -0.160696775
y: -0.688707829
z: -0.109716035
visibility: 0.999591053
, x: -0.160780147
y: -0.689005077
z: -0.110186279
visibility: 0.999690175
, x: -0.159020439
y: -0.690776
z: -0.0719697
visibility: 0.999475181
, x: -0.15854457
y: -0.691712
z: -0.0725662038
visibility: 0.999419689
, x: -0.158854142
y: -0.693026125
z: -0.0721746311
visibility: 0.999534726
, x: -0.0543245
y: -0.636023
z: -0.161656246
visibility: 0.999732792
, x: -0.0567518547
y: -0.66000241
z: -0.00516845472
visibility: 0.999396443
, x: -0.145553261
y: -0.624101758
z: -0.107771128
visibility: 0.999605715
, x: -0.14652513
y: -0.62935549
z: -0.0612656847
visibility: 0.999314189
, x: -0.0368569195
y: -0.438760161
z: -0.234108329
visibility: 0.999967217
, x: -0.0137652066
y: -0.47364074
z: 0.106940366
visibility: 0.999219656
, x: -0.109746411
y: -0.19489263
z: -0.270818591
visibility

KeyboardInterrupt: 